# Обработка файлов

Для извлечения данных из файлов, например, презентаций PowerPoint и PDF, можно изспользовать встроенные в GigaChain [загрузчики документов](/docs/modules/data_connection/document_loaders/).
Загрузчики преобразуют файлы в текст, который можно передать в модель.

## Парсинг на основе MIME-типов

:::note

Базовые примеры парсинга — в разделе, посвященном [загрузчикам документов](/docs/modules/data_connection/document_loaders/).

:::

В этом разделе показан пример парсинга на основе MIME-типа данных.
Этот способ полезен при разработке серверных приложений, которые могут принимать файлы от пользователей.
В таких случаях лучше всего предположить, что расширение файла, указанное пользователем, неверно, и вместо этого вывести mimetype из бинарного содержимого файла.

В примере используется HTML-файл, но представленный код должен работать и для других типов файлов.

In [1]:
import requests

response = requests.get("https://en.wikipedia.org/wiki/Car")
data = response.content
data[:20]

b'<!DOCTYPE html>\n<htm'

Настройка парсеров.

In [2]:
import magic
from langchain.document_loaders.parsers import BS4HTMLParser, PDFMinerParser
from langchain.document_loaders.parsers.generic import MimeTypeBasedParser
from langchain.document_loaders.parsers.txt import TextParser
from langchain_community.document_loaders import Blob

# Подготовка парсеров данных разных mime-типов
HANDLERS = {
    "application/pdf": PDFMinerParser(),
    "text/plain": TextParser(),
    "text/html": BS4HTMLParser(),
}

# Создание экземпляра парсера на основе mime-типов
MIMETYPE_BASED_PARSER = MimeTypeBasedParser(
    handlers=HANDLERS,
    fallback_parser=None,
)

mime = magic.Magic(mime=True)
mime_type = mime.from_buffer(data)

# Блоб представляет собой двоичные данные в виде ссылки (путь в файловой системе),
# либо в виде значения (байты в памяти).
blob = Blob.from_data(
    data=data,
    mime_type=mime_type,
)

parser = HANDLERS[mime_type]
documents = parser.parse(blob=blob)

In [3]:
print(documents[0].page_content[:30].strip())

Car - Wikipedia
